<div>
    <img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" ,="" align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">End-of-degree Project: Product Matching</p>
<p style="margin: 0; text-align:right;">Master's Degree in Data Science</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computer Science, Multimedia and Telecommunication Studies</p>
</div>

<h1>Product Final Data cleansing</h1>

<hr style="background-color:#BCBCBC"/>

In this Notebook...

<h2>Imports</h2>

In [15]:
import pandas as pd
import numpy as np

<h2>Matched Products Data Loading</h2>

In [26]:
PRODUCT_DATA_FINAL_PATH = '../RawData/ProductDataFinal.csv'

df = pd.read_csv(PRODUCT_DATA_FINAL_PATH)

print("Initial number of products (rows): {}".format(df.shape[0]))
print("Initial number of attributes (columns): {}".format(df.shape[1]))

Initial number of products (rows): 300000
Initial number of attributes (columns): 35


<h2>Attribute selection and processing</h2>

In [27]:
df = df.loc[:,
    [
        'WebShopName',
        'WebShopProductID',
        'BrandName',
        'Name',
        'Price',
        'Currency',
        'Color',
        'ProdanetID'
    ]]

df['WebShopName'] = df['WebShopName'].astype('category')
df['WebShopProductID'] = df['WebShopProductID'].astype('category')
df['BrandName'] = df['BrandName'].astype('category')
df['Name'] = df['Name'].astype('category')
df['Price'] = df['Price'].astype('float')
df['Currency'] = df['Currency'].astype('category')
df['Color'] = df['Color'].astype('category')
df['ProdanetID'] = df['ProdanetID'].astype('category')

<h2>Data filtering</h2>

In [28]:
# Work only with prices in EUR
df = df.loc[df['Currency'] == 'EUR']

# Shuffle the dataset
df = df.sample(frac=1)

In [29]:
df.head(5)

,WebShopName,WebShopProductID,BrandName,Name,Price,Currency,Color,ProdanetID
103617,Worten.pt,6089956,Epson,Projetor EPSON EB-680Wi,2799.00,EUR,NaN,1625873
244602,JPC.de,8152803,NaN,Omik K: Coño (Limited-Edition) (Box) auf 4 CDs...,39.99,EUR,NaN,1919939
150258,Amazon.de,B079JPNTGC,NaN,The Whiskey Bandit - Allein gegen das Gesetz [...,8.99,EUR,NaN,1903771
138128,JPC.de,1529506,NaN,"Charles Gounod: Faust (""Margarethe"") auf 2 DVD...",19.99,EUR,NaN,1391328
28619,Amazon.de,B00ISNGAQ4,NaN,Meine teuflischen Nachbarn [Blu-ray],8.84,EUR,NaN,1041560


In [30]:
df.describe(include = 'all')

,WebShopName,WebShopProductID,BrandName,Name,Price,Currency,Color,ProdanetID
count,296714,296714,191977,296711,296714.000000,296714,7826,296714.0
unique,784,194064,1824,199841,NaN,1,1852,159569.0
top,Amazon.de,5893697,Hama,Bosch HRG6753S2,NaN,EUR,schwarz,1845671.0
freq,75985,1321,48778,1322,NaN,296714,660,1322.0
mean,NaN,NaN,NaN,NaN,136.683255,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,806.697090,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,0.010000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,12.600000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,19.990000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,64.090000,NaN,NaN,NaN


<h2>Export</h2>

In [7]:
PRODUCT_DATA_FINAL_CLEAN_FILE_PATH = "../CleanData/CleanProductDataFinal.csv"

df.to_csv(PRODUCT_DATA_FINAL_CLEAN_FILE_PATH, sep=',', encoding='utf-8')

<h1>K Nearest Neighbors</h1>

<h2>Categorical variables encoding</h2>

In [11]:
X = pd.get_dummies(df.loc[:, ['WebShopName', 'BrandName']])
X = X.join(df['Price'])

y = df['ProdanetID'].values.tolist()

(296714, 2648)


AttributeError: 'list' object has no attribute 'shape'

In [14]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X, y)

MemoryError: 

In [ ]:
#%reset

In [49]:
# https://stackoverflow.com/questions/44862712/td-idf-find-cosine-similarity-between-new-document-and-dataset
# http://blog.christianperone.com/2013/09/machine-learning-cosine-similarity-for-vector-space-models-part-iii/

from sklearn.feature_extraction.text import TfidfVectorizer

documents = df['Name'].values.astype('U').tolist()

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print(tfidf_matrix.shape)

(296714, 126522)


In [99]:
from sklearn.metrics.pairwise import cosine_similarity

new_product_vectorized = tfidf_vectorizer.transform(["Lenovo ThinkPad T480 1.60GHz i5-8250U 14"" 1920 x 1080Pixeles Negro Portátil"])

similarity_matrix = cosine_similarity(new_product_vectorized, tfidf_matrix)

In [100]:
max_index = np.argmax(similarity_matrix)
max_prop = np.amax(similarity_matrix)

In [101]:
max_index, max_prop

(138566, 0.5788563189772609)

In [102]:
df.iloc[max_index]

WebShopName                                                  E_TEC.at
WebShopProductID                                               273429
BrandName                                                      Lenovo
Name                Lenovo ThinkPad T480, Core i5-8250U, 16GB RAM,...
Price                                                         1706.45
Currency                                                          EUR
Color                                                             NaN
ProdanetID                                                    2045891
Name: 68407, dtype: object